import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=12
btch=30
for i in range (1,cls_num+1):
    vars()['ecgH'+str(i)],vars()['ecg_tstH'+str(i)]=EOGHorizontalSignal(i)
os.chdir(cwd)
clses_lens=np.array([])
clses_lensH=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgH=np.array(ecgH1)
print(1,len(vars()['ecgH'+str(1)]), end='\t')
clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(1)]))
for i in range (2,cls_num+1):
    ecgH=np.concatenate((ecgH,vars()['ecgH'+str(i)]),axis=0)
    clses_lensH=np.append(clses_lensH,len(vars()['ecgH'+str(i)]))
    print( i,len(vars()['ecgH'+str(i)]), end='\t')

mx_sigH=max(clses_lensH)
print('\nshape ecgH=',np.shape(ecgH))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 446.32
min(train_feature_Altitude)= -1110.8
first_train_sample=
 [   1.         0.47769    0.60965 ... -110.76    -111.03    -110.9    ]
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	
shape ecgH= (362, 481)


In [4]:
#Vertical
for i in range (1,cls_num+1):
    vars()['ecgV'+str(i)],vars()['ecg_tstV'+str(i)]=EOGVerticalSignal(i)
os.chdir(cwd)
clses_lensV=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecgV=np.array(ecgV1)
print(1,len(vars()['ecgV'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecgV'+str(1)]))
for i in range (2,cls_num+1):
    ecgV=np.concatenate((ecgV,vars()['ecgV'+str(i)]),axis=0)
    clses_lensV=np.append(clses_lensV,len(vars()['ecgV'+str(i)]))
    print( i,len(vars()['ecgV'+str(i)]), end='\t')

mx_sigV=max(clses_lensV)
print('\nshape ecgV=',np.shape(ecgV))

first_train_shape= (362, 1251)
first_test_shape= (362, 1251)
classes_quantity= 12
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Count_labels= [30 30 30 30 30 31 31 30 30 30 30 30]
max(train_feature_Altitude)= 2540.7
min(train_feature_Altitude)= -923.79
first_train_sample=
 [   1.       -5.7032   -5.8955 ... -113.16   -112.85   -112.94  ]
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	
shape ecgV= (362, 481)


In [5]:
for i in range (1,cls_num+1):
    #vars()['ecgHV'+str(i)]=np.array([])
    vars()['ecg'+str(i)]=np.array(np.concatenate((vars()['ecgH'+str(i)][:,0:-1],vars()['ecgV'+str(i)]),axis=1))
    vars()['ecg_tst'+str(i)]=np.array(np.concatenate((vars()['ecg_tstH'+str(i)][:,0:-1],vars()['ecg_tstV'+str(i)]),axis=1))
ecg=np.array(np.concatenate((ecgH[:,:-1],ecgV),axis=1))

In [6]:
np.shape(ecg)

(362, 961)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [7]:
smpl_rte=960                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 30	2 30	3 30	4 30	5 30	6 31	7 31	8 30	9 30	10 30	11 30	12 30	
 max = 31


In [8]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(mx_wndws+15)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

46


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [10]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
for methd in range(60,80,10):
    for quality in range(1,7):
        if quality==1:
            sort=-5          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==2:
            sort=-4          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==3:
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==4:
            sort=-2          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==5:
            sort=-1          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        else :
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=1         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        for dp_slct_dstrb in range (1,6):#1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
            rprt=np.empty((0,3),float)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
                    vars()['ecg_aug'+str(i)+str(methd)]=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
                    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
                    #print("vlum_win=",vlum_win)
                    Hrzntl=srtd_dataH(methd,i,vlum_win,dp_slct_dstrb,sort)
                    Vrtcl=srtd_dataV(methd,i,vlum_win,dp_slct_dstrb,sort)
                    HrzntlVrtcl=np.concatenate((Hrzntl[:,:-1],Vrtcl),axis=1)
                    vars()['ecg_aug'+str(i)+str(methd)]=np.append(vars()['ecg_aug'+str(i)+str(methd)],HrzntlVrtcl,axis=0)
                    #print('aug shape=',np.shape(scl_rndm(ecg,i,vlum_win,mag=.1)))
                    #print('generated data shape for class ', i, ' = ', np.shape(vars()['ecg_aug'+str(i)+str(methd)]))

                #add augmented data to base data
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):   
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    xtrain=np.append(xtrain,vars()['ecg_aug'+str(i)+str(methd)],axis=0)
                    #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num+1):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                cls_wndws=np.array([])
                for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    cls_wndws=np.append(cls_wndws,wns)
                    #print(cls, wns, end='\t')
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
                #print('\n max instance in classes of test windows =', mx_wndws)

                #print('min tst =', np.min(wndws_test[:,:-1]))
                #print('max tst =', np.max(wndws_test[:,:-1]))

                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                #fullprint(ytest)

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                
                rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            maxm=rprt[np.argmax(rprt[:,1])]
            avrg=np.mean(rprt,axis=0)
            rslts=np.append(rslts,['mthd '+str(methd)+'  q '+str(quality)+'  Dist '+str(dp_slct_dstrb)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

12/12 [==============================] - 0s 3ms/step


In [11]:
len(vars()['wndws'+str(i)])

30

In [12]:
rslts

array(['mthd 60  q 1  Dist 1  average= [81 81 81]  max= [83 82 82]  var= 0',
       'mthd 60  q 1  Dist 2  average= [84 84 84]  max= [87 87 87]  var= 4',
       'mthd 60  q 1  Dist 3  average= [83 82 82]  max= [84 84 84]  var= 1',
       'mthd 60  q 1  Dist 4  average= [82 82 82]  max= [84 83 83]  var= 4',
       'mthd 60  q 1  Dist 5  average= [81 81 81]  max= [84 84 84]  var= 5',
       'mthd 60  q 2  Dist 1  average= [82 82 82]  max= [83 83 83]  var= 0',
       'mthd 60  q 2  Dist 2  average= [83 82 82]  max= [83 83 83]  var= 0',
       'mthd 60  q 2  Dist 3  average= [83 83 83]  max= [84 84 84]  var= 2',
       'mthd 60  q 2  Dist 4  average= [82 82 82]  max= [83 83 83]  var= 0',
       'mthd 60  q 2  Dist 5  average= [83 82 82]  max= [84 84 84]  var= 2',
       'mthd 60  q 3  Dist 1  average= [82 81 81]  max= [83 83 83]  var= 1',
       'mthd 60  q 3  Dist 2  average= [81 81 81]  max= [82 82 82]  var= 0',
       'mthd 60  q 3  Dist 3  average= [83 83 83]  max= [85 85 85]  var= 2',

In [13]:
np.savetxt(("Results_virgl.txt"),rslts,delimiter=',')
np.savetxt(("Results.csv"),rslts,delimiter='\n')

TypeError: Mismatch between array dtype ('<U66') and format specifier ('%.18e')